In [1]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from pathlib import Path

from tqdm.notebook import tqdm

import cv2 as cv
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision import models
from PIL import Image

from sklearn import metrics

import matplotlib.pyplot as plt

# Introduction

Magnetic resonance imaging is one

## Plan

1) 

In [17]:
class MRNet(nn.Module):
    def __init__(self, depth=3, width=128, dropout=0.2, base_model="resnet18"):
        """
        This constructor takes a pretrained resnet18 model and replaces the last layer by a custom neural net with
        four layers seperated by dropout layers.
        """
        super(MRNet, self).__init__()
        
        if base_model == "resnet18":
            self.model = models.resnet18(pretrained=True)
            nn_output = 512
        
        elif base_model == "resnet50":
            self.model = models.resnet50(pretrained=True)
            nn_output = 2048
        else:
            raise NotImplementedError
            
        for param in self.model.parameters():
            param.requires_grad = False

        # New nn
        dic = OrderedDict()
        for i in range(depth):
            if i==0:
                dic[f"lin{i+1}"] = nn.Linear(nn_output, width)
            else:
                dic[f"lin{i+1}"] = nn.Linear(width, width)
            dic[f'relu{i+1}'] = nn.ReLU()
            dic[f'drop{i+1}'] = nn.Dropout(dropout)
        dic[f'lin{i+2}'] = nn.Linear(width, 2)

        self.model.fc = nn.Sequential(dic)

    def forward(self, x):
        return self.model(x)


In [4]:
class MRIDataset(Dataset):
    def __init__(self, csv_file_path, root_dir, transform):
        # Saves the entire database : it is relatively small
        self.mri_database = pd.read_csv(csv_file_path)
        self.images = []
        self.labels = []

        for i in self.mri_database.index:
            image_path = root_dir / self.mri_database["image_path"][i]
            mask = self.mri_database["mask"][i]
            self.images.append(Image.open(image_path))
            self.labels.append(mask)

        self.labels = torch.tensor(self.labels)

        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.mri_database)

    def __getitem__(self, item):
        return self.transform(self.images[item]), self.labels[item]
    
    def show(self, item):
        im, label = self[item]
        im = im.permute(1, 2, 0).numpy() 
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        im = std * im + mean
        im = np.clip(im, 0, 1)

        plt.imshow(im)
        plt.title(f"Label: {'tumor' if label else 'no tumor'}")
        plt.axis('off')
        plt.show()



In [31]:
class MRNetTrainer:
#     model, optimizer, criterion, device, lr_scheduler, path_model_checkpoint, False
    def __init__(self, model, optimizer, criterion, device, lr_scheduler=None, 
                 path_best_model: Path = None, path_model_checkpoint: Path = None, from_checkpoint = True):
        self.path_model_checkpoint = path_model_checkpoint
        self.best_model_path = path_best_model
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        self.lr_scheduler = lr_scheduler

        self.epoch = 0

        self.loss = []
        self.metrics = []

        # Check if model already exists
        if from_checkpoint:
            if path_model_checkpoint.is_file():
                self.load_checkpoint()

    def save_checkpoint(self):
        checkpoint_data = {'epoch': self.epoch,
                           'model_state_dict': self.model.state_dict(),
                           'optimizer_state_dict': self.optimizer.state_dict(),
                           'loss': self.loss,
                           'metrics': self.metrics,
                           }

        torch.save(checkpoint_data, self.path_model_checkpoint)
        
    def _save_if_best(self):
        # "Best" defined by f1 score on "tumor" class detection
        recall = np.array([x["recall"][1] for x in  self.metrics])
        precision =  np.array([x["precision"][1] for x in  self.metrics])
        f_score = 2*recall*precision/(recall+precision + 1e-6)
        if f_score[-1] == np.max(f_score):
            torch.save({'model_state_dict': self.model.state_dict()}, self.best_model_path)
            
    def load_checkpoint(self):
        checkpoint = torch.load(self.path_model_checkpoint)

        self.epoch = checkpoint["epoch"]
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        self.loss = checkpoint["loss"]
        self.metrics = checkpoint["metrics"]
        
        print(f"Model updated using parameters from: {self.path_model_checkpoint}")

    def train(self, train_loader, validation_loader, n_epochs, verbose=False,):
        # Loop through epochs
        for epoch in tqdm(range(n_epochs),  desc="Epochs"):
            loss_sublist = []

            # Training
            self.model.train()
            for x, y in train_loader:
                self.optimizer.zero_grad()
                
                x, y = x.to(self.device), y.to(self.device)
                out = self.model(x)
                
                loss = self.criterion(out, y)
                loss.backward()
                self.optimizer.step()
                
                loss_sublist.append(loss.data.item())
                
            self.loss.append(np.mean(loss_sublist))
            if self.lr_scheduler:
                self.lr_scheduler.step()
            
            
            # Model evaluation
            self.model.eval()
            predictions = []
            ground_truths = []
            for x, y in validation_loader:
                x, y = x.to(self.device), y.to(self.device)
                out = self.model(x)
                predictions.append(out.cpu().data.numpy().argmax(axis=1))
                ground_truths.append(y.cpu().numpy())

            ground_truths = np.array(ground_truths).flatten()
            predictions = np.array(predictions).flatten()

            confusion_matrix = metrics.confusion_matrix(ground_truths, predictions, labels=[0, 1])

            eps = 1e-6  # To avoid zero divide
            accuracy = np.trace(confusion_matrix) / (np.sum(confusion_matrix) + eps)
            recall = np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=0) + eps)
            precision = np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=1) + eps)

            self.metrics.append({'accuracy': accuracy,
                                 'recall': recall,
                                 'precision': precision})
            self.epoch += 1
            self.save_checkpoint()
            self._save_if_best()
            
            if verbose:
                print("*"*25)
                print(f"Summary epoch {self.epoch}:")
                print(f"----Loss:     \t{self.loss[-1]:.3f} ")
                print(f"----Acc:      \t{self.metrics[-1]['accuracy']:.3f}")
                print(f"----Recall:   \t{np.round(self.metrics[-1]['recall'],3)}")
                print(f"----Precision:\t{np.round(self.metrics[-1]['precision'],3)}")
                print("----Confusion matrix:")
                print(confusion_matrix)

    def report(self, title=None):
        """
        Make a reporting with three plots :
            1) loss and accuracy VS epoch
            2) "No tumor" label: Recall VS Precision as a function of epoch
            3)  "Tumor"   label: Recall VS Presision as a function of epoch
        """
        
        los = self.loss
        acc = np.array([x['accuracy'] for x in self.metrics])
        rec = np.array([x['recall'] for x in self.metrics])
        pre = np.array([x['precision'] for x in self.metrics])

        fig, ax = plt.subplots(1, 3, figsize=(26/2.54, 8/2.54))
        color = 'tab:red'
        ax[0].plot(los, "-o", color=color)
        ax[0].set_ylabel("Loss")
        ax[0].set_xlabel("Epoch")

        ax2 = ax[0].twinx()
        color = 'tab:blue'
        ax2.set_ylabel('Accuracy', color = color,)  # we already handled the x-label with ax1
        ax2.plot(acc, "-o", color = color)
        ax2.tick_params(axis = 'y', color = color)

        ax[1].plot(rec[:,0], pre[:,0], label="No tumor")
        ax[1].scatter(rec[:,0], pre[:,0], c=np.linspace(0,1,self.epoch), cmap= "viridis")
        ax[1].set_xlim(0,1)
        ax[1].set_ylim(0,1)
        ax[1].set_xlabel("Recall")
        ax[1].set_ylabel("Precision")
        ax[1].legend()

        ax[2].plot(rec[:,1], pre[:,1], label="Tumor")
        ax[2].scatter(rec[:,1], pre[:,1],c=np.linspace(0,1,self.epoch), cmap= "viridis")
        ax[2].set_xlim(0,1)
        ax[2].set_ylim(0,1)
        ax[2].set_xlabel("Recall")
        ax[2].set_ylabel("Precision")
        ax[2].legend()
        
        if title:
            fig.suptitle(title)

        plt.tight_layout()
        plt.show()
        
        
        

In [12]:
root = Path("../input/mridata/")
path_db = Path("../input/mridata/database.csv")
path_model_checkpoint = Path("./model_checkpoint.pt")
path_best_model = Path("./model_best.pt")
# Hyperparameter selection

# NN Model
base_nn = "resnet50"
depth = 4
width = 256
dropout = 0.2

# Dataset
batch_size = 32

# optimizer : Adam
lr = 0.0002
betas = (0.9, 0,99)

# Algorithm
epochs = 25



In [7]:
# Dataset
# Data preprocessing and augmentation consistent with MRI data images
augmented_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=20, translate=(0.05, 0.05)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
mri_dataset = MRIDataset(path_db, root, augmented_transform)
mri_dataset.show(5)

In [13]:
# To avoid data leakage between the train_set and the valid_set between training sessions,
# DO NOT change the seed !
train_set_size = int(len(mri_dataset) * 0.85)
valid_set_size = len(mri_dataset) - train_set_size

train_set, valid_set = random_split(mri_dataset, [train_set_size, valid_set_size],
                                    generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=valid_set, batch_size=1)

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = MRNet(depth=depth, width=width, dropout=dropout, base_model= base_nn)
print(model.to(device))
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas)
# lr_scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size_up=5, mode="triangular2")
lr_scheduler = None
criterion = nn.CrossEntropyLoss()

mri_net_trainer = MRNetTrainer(model, optimizer, criterion, device, lr_scheduler, path_best_model, path_model_checkpoint, True)


In [35]:
mri_net_trainer.train(train_loader, validation_loader, 20, True)

In [36]:
mri_net_trainer.report()

In [50]:
def training_VS_parameters(name, depth, width, dropout, base_nn, mri_dataset, batch_size, lr, betas, epochs):
    path_model_checkpoint = Path(f"./{name}_model_checkpoint.pt")
    path_model_best = Path(f"./{name}_model_best.pt")
    
    # Model
    model = MRNet(depth, width, dropout, base_nn)
    model.to(device)
    
    # Dataset
    train_set_size = int(len(mri_dataset) * 0.85)
    valid_set_size = len(mri_dataset) - train_set_size
    train_set, valid_set = random_split(mri_dataset, [train_set_size, valid_set_size],
                                        generator=torch.Generator().manual_seed(42))
    
    train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(dataset=valid_set, batch_size=1)

    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas)
    criterion = nn.CrossEntropyLoss()

    mri_net_trainer = MRNetTrainer(model, optimizer, criterion, device, lr_scheduler, path_model_best, path_model_checkpoint, False)
    mri_net_trainer.train(train_loader, validation_loader, epochs, False)
    mri_net_trainer.report()
    
    with open(Path("./experiment_report.txt"), 'a') as datafile:
        datafile.write("-"*32+"\n")
        datafile.write(f"name: {name}\n")
        datafile.write(f"model_best: {path_model_best}\n")
        datafile.write(f"model_checkpoint: {path_model_checkpoint}\n")
        datafile.write(f"epochs: {epochs}\n")
        datafile.write(f"depth: {depth}\n")
        datafile.write(f"width: {width}\n")
        datafile.write(f"dropout: {dropout}\n")
        datafile.write(f"batch_size: {batch_size}\n")
        datafile.write(f"learning_rate: {lr}\n")
        datafile.write(f"betas: {betas}\n")
        
        acc = np.array([x["accuracy"] for x in mri_net_trainer.metrics])
        rec = np.array([x["recall"] for x in mri_net_trainer.metrics])
        pre = np.array([x["precision"] for x in mri_net_trainer.metrics])
        
        index_best = np.argmax(rec[:,1])
        
        datafile.write(f"loss: {mri_net_trainer.loss[index_best]}\n")
        datafile.write(f"accuracy: {acc[index_best]}\n")
        datafile.write(f"recall: {tuple(np.round(rec[index_best], 3))}\n")
        datafile.write(f"precision: {tuple(np.round(pre[index_best],3))}\n")
        
    return mri_net_trainer


In [86]:
model_data = torch.load(path_best_model)
model = MRNet(depth=depth, width=width, dropout=dropout, base_model= base_nn)
model.load_state_dict(model_data["model_state_dict"])
model.to(device)
model.eval()

i=0
fig, ax = plt.subplots(1, 5, figsize=(15,10), dpi=100)
act = nn.Softmax(0)
for x, y in validation_loader:
    x = x.to(device)
    out = model(x).cpu().detach().view(-1)
    z = act(out)
    y_hat = torch.argmax(z)
    prob = z[y_hat]
    
    if y_hat != y:
        im = x.cpu()[0]
        im = im.permute(1, 2, 0).numpy() 
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        im = std * im + mean
        im = np.clip(im, 0, 1)

        ax[i].imshow(im)
        ax[i].set_title(f"Label: {'tumor' if y.numpy()[0] else 'no tumor'}\n Pred: {'tumor' if y_hat.numpy() else 'no tumor'} ({int(prob*100)}%)")
        ax[i].axis('off')
        i+=1
        
    if i>=5:
        plt.show()
        break


In [47]:
training_VS_parameters(name="base", depth=3, width=128, dropout=0.2, base
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [51]:
training_VS_parameters(name="lr-5e-3", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.9, 0.999), epochs=25)

In [52]:
training_VS_parameters(name="lr-2e-4", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=2e-4, betas=(0.9, 0.999), epochs=25)

In [53]:
training_VS_parameters(name="betas-09-099", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.9, 0.99), epochs=25)

In [54]:
training_VS_parameters(name="betas-09-09999", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.9, 0.9999), epochs=25)

In [55]:
training_VS_parameters(name="betas-75-0999", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.75, 0.999), epochs=25)

In [56]:
training_VS_parameters(name="betas-095-0999", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.95, 0.999), epochs=25)

In [57]:
training_VS_parameters(name="depth-2", depth=2, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.9, 0.999), epochs=25)

In [58]:
training_VS_parameters(name="depth-4", depth=4, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=5e-3, betas=(0.9, 0.999), epochs=25) 

In [59]:
training_VS_parameters(name="batch-25", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=25, 
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [60]:
training_VS_parameters(name="batch-100", depth=3, width=128, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=100,
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [61]:
training_VS_parameters(name="width-64", depth=3, width=64, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [62]:
training_VS_parameters(name="width-256", depth=3, width=256, dropout=0.2, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [63]:
training_VS_parameters(name="dropout-03", depth=3, width=128, dropout=0.3, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [64]:
training_VS_parameters(name="dropout-01", depth=3, width=128, dropout=0.1, 
                       mri_dataset=mri_dataset, batch_size=50, 
                       lr=1e-3, betas=(0.9, 0.999), epochs=25)

In [65]:
!jupyter nbconvert --to html your_notebook_name.ipynb